In [ ]:
import numpy as np
import signalflow as sf
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Create app
app = App()

In [2]:
# load an image
img_path = "images/cellular_dataset/Timepoint_005_220518-ST_C03_s1.jpg"
# img_path = "images/test.jpg"
img = app.load_image(img_path)

In [ ]:
# Create objects
mean_pix = MeanPixelValue()
# mean_pix = MeanPixelValue(channels=2) # test RGB
theremin = Theremin()
# theremin = Theremin(np.linspace(220, 440, 64).tolist(), panning=[-1, 1]) # test multichannel
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

In [ ]:
app.features

In [ ]:
app.attach_feature(mean_pix)
app.features

In [ ]:
app.synths

In [ ]:
app.attach_synth(theremin)
app.synths

In [ ]:
app.mappers

In [ ]:
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
app.detach_mapper(pix2freq)
app.mappers

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)

In [ ]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
# detach a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# detach old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# Non-Real-Time Rendering

In [ ]:
# example: horizontal scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]

target_filename = "horizontal_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

In [ ]:
# example: vertical scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 500,
        "probe_height": 1,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_y": 499
    })
]

target_filename = "vertical_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

# Envelope system proto

In [1]:
import numpy as np
import signalflow as sf
import time
import random

In [ ]:
# create a graph
graph = sf.AudioGraph.get_shared_graph()
if graph is not None:
    graph.destroy()
config = sf.AudioGraphConfig()
config.output_buffer_size = 480
graph = sf.AudioGraph(config)
graph.status

In [3]:
sine = sf.SineOscillator(440)
env = sf.ADSREnvelope(
    attack=0.01,
    sustain=0.8,
    release=0.5,
    gate=0
)
out = sine * env

graph.play(out)

In [4]:
env.set_input("gate", 1)

In [5]:
env.set_input("gate", 0)

In [3]:
class SineSynth(sf.Patch):
    def __init__(self):
        super().__init__()
        frequency = self.add_input("frequency", 440)
        gate = self.add_input("gate", 0)
        amp = self.add_input("amplitude", 0.5)
        pan = self.add_input("panning", 0)
        release = self.add_input("release", 0.5)
        sine = sf.SineOscillator(frequency)
        env = sf.ADSREnvelope(
            attack=0.01,
            decay=0.1,
            sustain=0.5,
            release=release,
            gate=gate
        )
        stereo = sf.StereoPanner(sine * env * amp, pan=pan)
        self.set_output(stereo)

In [130]:
class SineSynth2(sf.Patch):
    def __init__(self):
        super().__init__()
        frequency = self.add_input("frequency", 440)
        amp = self.add_input("amplitude", 0.5)
        pan = self.add_input("panning", 0)
        release = self.add_input("release", 0.5)
        sine = sf.SineOscillator(frequency)
        env = sf.ASREnvelope(
            attack=0.01,
            sustain=0.5,
            release=release,
            curve=2,
            clock=0
        )
        stereo = sf.StereoPanner(sine * env * amp, pan=pan)
        self.set_trigger_node(env)
        self.set_output(stereo)

In [131]:
my_synth = SineSynth2()
# my_synth.auto_free = True
graph.play(my_synth)


In [133]:
my_synth.trigger()

In [17]:
my_synth.set_input("gate", 1)

In [18]:
my_synth.set_input("gate", 0)

In [19]:
graph.stop(my_synth)

In [ ]:
graph.status

In [21]:
graph.clear()

In [30]:
patch = SineSynth2()
spec = patch.to_spec()

In [35]:
n_hits = 100
for i in range(n_hits):
    voice = SineSynth2()
    voice.auto_free = True
    voice.set_input("frequency", random.uniform(220, 880))
    voice.set_input("amplitude", random.uniform(0.01, 0.02))
    voice.set_input("panning", random.uniform(-1, 1))
    voice.set_input("release", random.uniform(0.01, 0.1))
    voice.play()
    voice.trigger()
    time.sleep(0.1)

# Linear Smoothing Proto

In [9]:
class LinearSmooth(sf.Patch):
    def __init__(self, input_sig, smooth_time=0.1):
        super().__init__()
        graph = sf.AudioGraph.get_shared_graph()
        samps = graph.sample_rate * smooth_time
        steps = samps / graph.output_buffer_size
        steps = sf.If(steps < 1, 1, steps)

        current_value_buf = sf.Buffer(1, graph.output_buffer_size)
        current_value = sf.FeedbackBufferReader(current_value_buf)

        history_buf = sf.Buffer(1, graph.output_buffer_size)
        history = sf.FeedbackBufferReader(history_buf)

        change = input_sig != history
        target = sf.SampleAndHold(input_sig, change)
        diff = sf.SampleAndHold(target - current_value, change)

        increment = diff / steps

        out = sf.If(sf.Abs(target - current_value) < sf.Abs(increment), target, current_value + increment)
        graph.add_node(sf.HistoryBufferWriter(current_value_buf, out))
        graph.add_node(sf.HistoryBufferWriter(history_buf, input_sig))
        self.set_output(out)

In [ ]:
# test it
freq = sf.Constant(440)
interp_time = sf.Constant(0.1)
test_ramp = LinearSmooth(freq, interp_time)
sine = sf.SineOscillator(test_ramp)

In [ ]:
graph.play(sine)

In [ ]:
graph.output_buffer_size

In [27]:
freq.set_value(220)

In [26]:
freq.set_value(880)

In [24]:
interp_time.set_value(0.1)

In [134]:
graph.stop(sine)

In [6]:
graph.clear()

In [ ]:
graph.status

# Envelope UI proto

In [43]:
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Label, Layout, Box, VBox, HBox, GridBox, Button, IntSlider, FloatSlider, FloatLogSlider, ToggleButton, Accordion, Text, BoundedFloatText, FloatText
from IPython.display import display
from pixasonics.ui import find_widget_by_tag, Model

In [44]:
class ADSRCanvas():
    def __init__(self, width=200, height=100, parent_ui=None):
        self.width = width
        self.height = height
        self.parent_ui = parent_ui
        self.canvas = Canvas(width=width, height=height)
        self.draw()

    def __call__(self):
        return self.canvas

    @property
    def attack(self):
        return self.parent_ui.attack if self.parent_ui is not None else 1

    @property
    def decay(self):
        return self.parent_ui.decay if self.parent_ui is not None else 1

    @property
    def sustain(self):
        return self.parent_ui.sustain if self.parent_ui is not None else 1

    @property
    def release(self):
        return self.parent_ui.release if self.parent_ui is not None else 1

    def draw(self):
        total_length = self.attack + self.decay + self.release
        length_coeff = self.width / total_length
        with hold_canvas(self.canvas):
            self.canvas.clear()
            self.canvas.stroke_style = "black"
            self.canvas.line_width = 2
            self.canvas.begin_path()
            # move to the bottom left corner
            self.canvas.move_to(0, self.height)
            # draw the attack
            self.canvas.line_to(self.attack * length_coeff, 0)
            # draw the decay
            self.canvas.line_to((self.attack + self.decay) * length_coeff, (1 - self.sustain) * self.height)
            # draw the release
            self.canvas.line_to(self.width, self.height)
            self.canvas.stroke()
            # put a red circle at each break-point
            self.canvas.stroke_style = "red"
            # at attack point
            self.canvas.stroke_circle(0, self.height, 3)
            # at decay point
            self.canvas.stroke_circle(self.attack * length_coeff, 0, 3)
            # at sustain point
            self.canvas.stroke_circle((self.attack + self.decay) * length_coeff, (1 - self.sustain) * self.height, 3)
            # at release point
            self.canvas.stroke_circle(self.width, self.height, 3)


In [ ]:
my_adsr = ADSRCanvas()
my_adsr()

In [ ]:
params = {
    "attack": {
        "min": 0.001,
        "max": 3600,
        "default": 0.01,
        "step" : 0.01,
        "param_name": "attack"
    },
    "decay": {
        "min": 0.001,
        "max": 3600,
        "default": 0.01,
        "step" : 0.01,
        "param_name": "decay"
    },
    "sustain": {
        "min": 0,
        "max": 1,
        "default": 0.5,
        "step": 0.1,
        "param_name": "sustain"
    },
    "release": {
        "min": 0.001,
        "max": 3600,
        "default": 0.1,
        "step" : 0.1,
        "param_name": "release"
    }
}

class EnvelopeCard():
    def __init__(
            self,
            name: str = "Envelope", 
            id: str = "# ID",
            params: dict = {},
    ):
        self.name = name
        self.id = id
        self.params = params
        self.app = None
        self.envelope = None

        # private attributes
        self._attack = Model(params["attack"]["default"])
        self._decay = Model(params["decay"]["default"])
        self._sustain = Model(params["sustain"]["default"])
        self._release = Model(params["release"]["default"])

        self.create_ui()
        self.update()

    @property
    def attack(self):
        return self._attack.value
    
    @attack.setter
    def attack(self, value):
        self._attack.value = value
        self.update()

    @property
    def decay(self):
        return self._decay.value
    
    @decay.setter
    def decay(self, value):
        self._decay.value = value
        self.update()

    @property
    def sustain(self):
        return self._sustain.value

    @sustain.setter
    def sustain(self, value):
        self._sustain.value = value
        self.update()

    @property
    def release(self):
        return self._release.value
    
    @release.setter
    def release(self, value):
        self._release.value = value
        self.update()

    def update_duration(self):
        duration = find_widget_by_tag(self.card, "duration")
        duration.value = round(self.attack + self.decay + self.release, 4)

    def update(self):
        if self.envelope is not None:
            self.envelope.set_param_from_ui("attack", self.attack)
            self.envelope.set_param_from_ui("decay", self.decay)
            self.envelope.set_param_from_ui("sustain", self.sustain)
            self.envelope.set_param_from_ui("release", self.release)
        self.update_duration()
        self.canvas.draw()

    def __call__(self):
        return self.card
    
    def create_ui(self):
        self.create_card()
        self.bind_models()

    def bind_models(self):
        attack = find_widget_by_tag(self.card, "attack")
        self._attack.bind_widget(attack, extra_callback=self.update)

        decay = find_widget_by_tag(self.card, "decay")
        self._decay.bind_widget(decay, extra_callback=self.update)

        sustain = find_widget_by_tag(self.card, "sustain")
        self._sustain.bind_widget(sustain, extra_callback=self.update)

        release = find_widget_by_tag(self.card, "release")
        self._release.bind_widget(release, extra_callback=self.update)

    
    def create_card(self):
        envelope_label = Label(
            value=self.name, 
            style=dict(
                font_weight='bold',
                font_size='20px'))

        envelope_id = Label(
            value="#" + self.id, 
            style=dict(
                font_weight='bold',
                font_size='10px',
                text_color='gray'))

        top_block = Box(
            [envelope_label, envelope_id], 
            layout=Layout(justify_content='space-between'))

        param_boxes = []
        for param, param_dict in params.items():
            value = param_dict["default"]
            param_label = Label(value=param.capitalize())
            param_numbox = BoundedFloatText(
                value=value,
                step=param_dict["step"],
                min=param_dict["min"],
                max=param_dict["max"],
                layout=Layout(width='90%'))
            param_numbox.tag = param
            param_box = VBox(
                [param_label, param_numbox], 
                layout=Layout(
                    justify_content='space-between',
                    height='100%'
                    )
            )
            param_boxes.append(param_box)
        params_display = HBox(
            param_boxes,
            layout=Layout(
                justify_content='space-between',
                width='100%'
            )
        )
        self.canvas = ADSRCanvas(400, 200, self)
        self.canvas.tag = "canvas"
        canvas_wrapper = VBox(
            [self.canvas()],
            layout=Layout(
                justify_content='center',
                width='100%',
                padding='4px'
            )
        )

        duration_label = Label(value="Duration:")
        duration_value = FloatText(
            value=0,
            disabled=True,
            layout=Layout(width='auto'))
        duration_value.tag = "duration"
        duration_box = HBox(
            [duration_label, duration_value], 
            layout=Layout(
                justify_content='flex-start',
                width='100%'
            )
        )

        self.card = VBox(
            [top_block, params_display, canvas_wrapper, duration_box],
            layout=Layout(
                justify_content='space-between',
                border='1px solid black',
                min_width='280px',
                max_width='500px',
                min_height='240px',
                max_height='400px',
                margin='5px',
                padding='5px'
            )
        )

test_card = EnvelopeCard("Envelope", "001", params)
display(test_card())

In [47]:
test_card.attack = 0.01
test_card.decay = 0.01
test_card.sustain = 0.8
test_card.release = 2

In [48]:
import signalflow as sf

In [134]:
class Envelope(sf.Patch):
    def __init__(self):
        super().__init__()
        self.params = {
            "attack": {
                "min": 0.001,
                "max": 3600,
                "default": 0.01,
                "step" : 0.01,
                "param_name": "attack"
            },
            "decay": {
                "min": 0.001,
                "max": 3600,
                "default": 0.01,
                "step" : 0.01,
                "param_name": "decay"
            },
            "sustain": {
                "min": 0,
                "max": 1,
                "default": 0.5,
                "step": 0.1,
                "param_name": "sustain"
            },
            "release": {
                "min": 0.001,
                "max": 3600,
                "default": 0.1,
                "step" : 0.1,
                "param_name": "release"
            }
        }
        for param in self.params.keys():
            self.params[param]["value"] = self.params[param]["default"]

        gate = self.add_input("gate", 0)
        attack = self.add_input("attack", self.params["attack"]["default"])
        decay = self.add_input("decay", self.params["decay"]["default"])
        sustain = self.add_input("sustain", self.params["sustain"]["default"])
        release = self.add_input("release", self.params["release"]["default"])

        adsr = sf.ADSREnvelope(
            attack=attack,
            decay=decay,
            sustain=sustain,
            release=release,
            gate=gate
        )

        asr = sf.ASREnvelope(
            attack=attack,
            sustain=sustain,
            release=release,
            clock=0
        )

        self.set_trigger_node(asr)
        self.set_output(adsr + asr)

        # self.set_output(self.adsr)

        self.id = str(id(self))
        self.create_ui()

    def on(self):
        self.set_input("gate", 1)

    def off(self):
        self.set_input("gate", 0)

    def __getitem__(self, key):
        return self.params[key]
    
    def create_ui(self):
        self._ui = EnvelopeCard("Envelope", self.id, self.params)
        self._ui.envelope = self

    @property
    def ui(self):
        return self._ui()
    
    def set_param_from_ui(self, param_name, value):
        self.params[param_name]["value"] = value
        self.set_input(param_name, value)
    
    @property
    def attack(self):
        return self.params["attack"]["value"]
    
    @attack.setter
    def attack(self, value):
        self.params["attack"]["value"] = value
        self.set_input("attack", value)
        self._ui.attack = value

    @property
    def decay(self):
        return self.params["decay"]["value"]
    
    @decay.setter
    def decay(self, value):
        self.params["decay"]["value"] = value
        self.set_input("decay", value)
        self._ui.decay = value

    @property
    def sustain(self):
        return self.params["sustain"]["value"]
    
    @sustain.setter
    def sustain(self, value):
        self.params["sustain"]["value"] = value
        self.set_input("sustain", value)
        self._ui.sustain = value

    @property
    def release(self):
        return self.params["release"]["value"]
    
    @release.setter
    def release(self, value):
        self.params["release"]["value"] = value
        self.set_input("release", value)
        self._ui.release = value

In [135]:
my_env = Envelope()

In [ ]:
my_env.ui

In [55]:
my_env.sustain = 0.1

In [ ]:
my_env.attack, my_env.decay, my_env.sustain, my_env.release

In [139]:
testsine = sf.SineOscillator(440)
testenv = Envelope()
test_bus = sf.Bus(1)
test_bus.add_input(testenv.output)
out = testsine * test_bus

graph.play(out)

In [151]:
testenv.on()

In [152]:
testenv.off()

In [149]:
testenv.trigger()

In [109]:
graph.stop(out)

In [137]:
graph.clear()

In [ ]:
graph.status